<a href="https://colab.research.google.com/github/pikrol/Web_crawling/blob/main/Web_crawling_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install anytree

     |████████████████████████████████| 41 kB 281 kB/s 


In [3]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from collections import defaultdict
from anytree import Node, RenderTree

In [65]:
def print_website_structure(link, internal_only = True):

  page = requests.get(link) #Get page HTML
  page_soup = BeautifulSoup(page.text, 'html.parser') #Parse HTML
  all_a = page_soup.findAll('a') #Find all 'a' tags
  all_paths = list()

  # (1) Prepare a list of all paths
  for a in all_a:

      if 'href' in a.attrs:

        href = a.attrs["href"] #Get href attribute value
        href = href[len('https://'): ] #Remove https prefix
        path = href.split('/') #Create list with subsequent pages
        path = list(filter(None, path)) #Remove empty strings

        #Prepare condition for storing the path
        if internal_only:
          append = path not in all_paths and len(path) > 0 and path[0] == link[len('https://'): ]
        else:
          append = path not in all_paths and len(path) > 0

        #Store the path
        if append: 
          all_paths.append(path)

  # (2) Get pairs of subsequent pages
  all_pairs = list()
  for path in all_paths:
    for i in range(len(path)):
      if len(path) - 1 == i:
        pair = [path[i]]
      else:
        pair = [path[i], path[i+1]]
      if pair not in all_pairs: all_pairs.append(pair)

  # (3) Prepare the tree and render it
  def add_nodes(nodes, parent, child):
      if parent not in nodes:
          nodes[parent] = Node(parent)  
      if child not in nodes:
          nodes[child] = Node(child)
      nodes[child].parent = nodes[parent]

  #Prepare df from pairs
  data = pd.DataFrame(columns=["Parent","Child"], data = all_pairs)

  #Create all nodes
  nodes = {} 
  data.apply(lambda x: add_nodes(nodes, x["Parent"], x["Child"]), axis=1)

  #Find root
  root = data[~data["Parent"].isin(data["Child"])]["Parent"].unique()[0] 

  #Print the tree
  for pre, _, node in RenderTree(nodes[root]):
      print("%s%s" % (pre, node.name))

In [66]:
print_website_structure('https://multilada.pl')

multilada.pl
├── projekty
│   ├── starwords
│   ├── cdi
│   ├── polkanorski
│   └── clt
├── zespol
├── kontakt
├── starwords-faq
├── polityka-prywatnosci
├── polityka-prywatnosci-polkanorski
├── en
└── no
    └── None
